<a href="https://colab.research.google.com/github/pGaldino25/projeto_imersao_ia_alura/blob/pGaldino25%2Fmain/my_first_LLM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [22]:
!pip install -q -U google-generativeai

In [20]:
# Import the Python SDK
import google.generativeai as genai
# Used to securely store your API key
from google.colab import userdata

GOOGLE_API_KEY=userdata.get('projeto_alura')
genai.configure(api_key=GOOGLE_API_KEY)

In [ ]:
import textwrap

from openpyxl import load_workbook
from IPython.display import display
from IPython.display import Markdown
from google.generativeai import generate_text

# Carregue o arquivo .xlsx
workbook = load_workbook(filename="/content/Planilha_Controle_Orcamento.xlsx")

# Selecione a planilha desejada
sheet = workbook.active

# Extraia os dados da planilha
data = []
for row in sheet.iter_rows():
    row_data = [cell.value for cell in row]
    data.append(row_data)

# Converta os dados em texto
text_data = "\n".join([",".join(str(cell) for cell in row) for row in data])

generation_config = {
    "candidate_count": 1,
    "temperature": 0.5,
}

safety_settings = {
    "HARASSMENT": "BLOCK_NONE",
    "HATE": "BLOCK_NONE",
    "SEXUAL": "BLOCK_NONE",
    "DANGEROUS": "BLOCK_NONE"
}

model = genai.GenerativeModel(model_name='gemini-1.5-pro-latest',
                              generation_config=generation_config,
                              safety_settings=safety_settings)



perguntas = [
    'Quais são seus objetivos financeiros? ',
    'Você considera algum gasto essencial e inegociável? Por exemplo, plano de saúde: ',
    'Como você se sente com seus gastos em cada categoria? Tem alguma que te deixa particularmente preocupado(a)? ',
    'Gostaria de focar em economizar em alguma área específica? Habitação, lazer, cartões? '
]

respostas = []

for pergunta in perguntas:
    resposta = input(pergunta)
    respostas.append(resposta)

prmpt1, prmpt2, prmpt3, prmpt4 = respostas

prompt = f'''
Tenho essa planilha de orçamento financeiro familiar,
Me indique onde eu estou gastando muito dinheiro e
me dê sugestões de como melhorar e economizar dinheiro.
Objetivos financeiros: {prmpt1}.
Gasto essencial e inegociável: {prmpt2}.
Tenho o seguinte sentimento com gastos na categoria: {prmpt3}.
Gostaria de focar em economizar na área: {prmpt4}'''
contents = [text_data, prompt]
# Imprima a resposta do modelo
response = model.generate_content(contents)

def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

display(to_markdown(response.text))